# Optimization

Tests to determine memory allocation and performance of varaious functions, looks for bottle necks. 

## Quadrature

### Updating quadrature scheme

In [139]:
## initialize quadrature
include("MvGaussHermite.jl")
m = 5
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)

# test updating 
MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0]) end


  0.000226 seconds (1.76 k allocations: 157.500 KiB)


In [140]:
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])end


  0.000238 seconds (1.22 k allocations: 118.594 KiB)


### Evaluating quadrature

In [155]:
include("MvGaussHermite.jl")
m = 10
f  = x -> sin(x[1]*x[2])
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.expected_value(f, Quad)
@time MvGaussHermite.expected_value(f, Quad)

  0.000034 seconds (16 allocations: 1.938 KiB)


1.6263032587282567e-18

## Updating beleif states
